In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
from training_functions import *

In [2]:
path='../outputs/'
optim_results = get_data(name_sav='optimization_results.sav', path=path)

# 1. AJUSTE FINAL.

El ajuste final de modelo optimizado es realizado por la función __training_model__ contenida en el script __training_functions.py__. Esta función se encarga también de:
1. guardar el modelo final en un archivo binario para su consumo,
2. generar las métricas de __testeo__, las cuales se obtienen al aplicar el modelo al conjunto de prueba,
3. generar gráficos de interés como el histograma de probabilidades de testeo,
4. calcular las features importances.

Los parámetros que esta función necesita son ya todos conocidos, a excepción de dos:

In [3]:
# Binario para el cálculo de las feature importances.
with_feature_importances=False
# Binario para la construcción del histograma de probabilidades de testeo.
with_probability_density=False

__¡¡¡ADVERTENCIA !!!:__ El cálculo de las feature importances para `HistGradientBoostingClassifier`
tarda horas. Esto debido a que se calculan vía shap-values, puesto que `scikit-learn`no las ofrece nativamente para este estimador.

In [4]:
name_train_sav='wA_train.sav'
name_test_sav='wA_test.sav'
features_names =  ['Location',
                   'MinTemp',
                   'Rainfall',
                   'WindGustDir',
                   'WindGustSpeed',
                   'WindDir9am',
                   'WindDir3pm',
                   'WindSpeed3pm',
                   'Humidity9am',
                   'Humidity3pm', 
                   'Pressure9am',
                   'RainToday', 
                   'month',
                   'week_of_year']
objective_name = 'RainTomorrow'
path='../outputs/'

Se procede a ajustar el modelo final.

In [5]:
train_results = training_model(best_hyper_info=optim_results,
                               name_train_sav=name_train_sav,
                               name_test_sav=name_test_sav,
                               features_names=features_names,
                               objective_name=objective_name,
                               with_feature_importances=with_feature_importances,
                               with_probability_density=with_probability_density,
                               path=path)

2023-07-03 19:31:16 INFO: PROCESO DE ENTRENAMIENTO Y GUARDADO DE MODELO.
2023-07-03 19:31:16 INFO: CONFIGURACIÓN DE DATA.
2023-07-03 19:31:16 INFO: TUBERÍAS PARA PROCESAMIENTO Y MUESTREO.
2023-07-03 19:31:16 INFO: AJUSTE FINAL.
2023-07-03 19:32:32 INFO: FIN DE AJUSTE FINAL.


# 2. COMPARACIÓN ENTRE TESTEO Y CV

In [18]:
from IPython.display import display

In [19]:
path='../outputs/'
train_results = get_data(name_sav='final_model_info_RainTomorrow_1.sav', path=path)

Veamos la estabilidad del modelo al comparar las métricas de __cross-validation__ y de __testeo__.

In [21]:
d_f1 = train_results[['cv_f1', 'final_f1_score']]
d_rec = train_results[['cv_recall', 'final_recall']]
d_prec =  train_results[['cv_precision', 'final_precision']]
d_m_c = train_results[['cv_m_c', 'final_matthews_corrcoef']]
display(d_f1, d_rec, d_prec, d_m_c)

,cv_f1,final_f1_score
0,0.63863,0.631234


,cv_recall,final_recall
0,0.785916,0.776993


,cv_precision,final_precision
0,0.53788,0.531523


,cv_m_c,final_matthews_corrcoef
0,0.525794,0.510783


Se puede observar que las métricas son similares entre ellas, lo cual se puede considerar como "una buena señal" y sugiere que el modelo generaliza de manera razonable para datos no vistos.

# 3. SOBRE LAS MÉTRICAS DE TESTEO

Estas son las métricas asociadas al conjunto de prueba.

In [30]:
train_results[['final_recall', 'final_precision',
       'final_accuracy', 'final_f1_score', 'final_roc_auc',
       'final_matthews_corrcoef', 'final_recall_maj', 'final_precision_maj']]

,final_recall,final_precision,final_accuracy,final_f1_score,final_roc_auc,final_matthews_corrcoef,final_recall_maj,final_precision_maj
0,0.776993,0.531523,0.792169,0.631234,0.786834,0.510783,0.796674,0.923268


De las métricas de __testeo__ se dan algunas observaciones:
1. __`final_recall`__ `= 0.776993`, quiere decir que la gran mayoría de muestras pertenecientes a la clase minoritaria, es decir, "SI" va a llover mañana, son clasificadas correctamente.
2. __`final_precision`__ `= 0.531523`, quiere decir que si bien se localiza correctamente a la gran mayoría de la clase minoritaria, "el precio que hay que pagar" para reconocerlos es el de tener a muchos falsos positivos. De hecho, se puede decir que del total de muestras que el modelo dice que son "SI", en realidad `(1 - 0.531523)% = 46.8447%` NO LO SON.
3. __`final_f1_score`__ `= 0.631234`, quiere decir que hay un equilibrio pobre entre __`recall`__ y __`precision`__. Algo similar puede concluirse observando el __`final_matthews_corrcoef`__.
4.   __`final_recall_maj`__ `= 0.796674` y __`final_precision_maj`__ `= 0.923268`, son las contrapartes de __`recall`__ y __`precision`__ pero para la clase mayoritaria. Naturalmente, a la clase mayoritaria le va bien.
4. __`final_roc_auc`__ `= 0.786834`, es el área bajo la curva ROC. Informa que el modelo clasifica correctamente a la mayoría tanto en la clase minoritaria como en la mayoritaria (observar los respectivos __`recalls`__), pero no toma en cuenta el escenario de desequilibro extremo cuya influencia se ve reflada en el __`final_precision`__.

# 4. COMENTARIOS FINALES

1. Los scores de decisión, interpretados como "probabliddades", que brindan algunos algoritmos de clasificación como el `HistGradientBoostingClassifier`, muchas veces aun no pueden interpretarse como verdaderas probabilidades. Necesitan pasar por un proceso de ajuste. Siendo así, queda pendiente revisar si el modelo final está __calibrado__. 
2. El "modelo final", más bien el objeto conformado por una __Pipiline__ de procedimientos cuyo paso final es el estimador ajustado, ya tiene integrado procesos como la codificación de variables categóricas.
3. Pueden consultarse los gráficos obtenidos en la carpeta __outputs/__. Sus nombres son:
    * __final_feature_importances_RainTomorrow_1.html__: exhibe el histograma en orden descendente de las feature importances. Al igual que con la __información mutua__, la variable `Humidity3pm` ocupa el primer lugar de importancia.
    * __final_density_RainTomorrow_1.html__: exhibe el histograma de las probabilidades de testeo.
4. Se decide, por el momento, llevar a producción al modelo final actual a reserva de mejorar su rendimiento, sobreto enfocando los esfuerzos para mejorar el __f1__, y más precisamente, el __precision__.